In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

np.random.seed(0)

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
prop_cycle = plt.rcParams['axes.prop_cycle']

In [ ]:
def F(u):
    return (u**2 - 1)**2/4

def f(u):
    return u - u**3

## Plot Double Well Potential

In [ ]:
u = np.linspace(-2, 2, 100)
plt.plot(u, F(u))
plt.xlabel("$u$")
plt.ylabel("$F(u)$")
plt.title("Double Well Potential")
plt.savefig("double-well-potential.pdf", bbox_inches="tight")
plt.show()

## Allen-Cahn Equation

### Initial Condition

Here we choose $u_0(x; \mu) = \mu_1x(x-1)(x-\mu_2)$ with $\mu_1, \mu_2 \in [0.1, 0.9]$.

In [ ]:
def u0(x, mu):
    return mu[0]*x*(x-1)*(x-mu[1])

In [ ]:
# plot some examples of u0
x = np.linspace(0, 1, 100)
plt.plot(x, u0(x, [0.1, 0.1]), label="0.1, 0.1")
plt.plot(x, u0(x, [0.1, 0.9]), label="0.1, 0.9")
plt.plot(x, u0(x, [0.5, 0.5]), label="0.5, 0.5")
plt.plot(x, u0(x, [0.9, 0.1]), label="0.9, 0.1")
plt.plot(x, u0(x, [0.9, 0.9]), label="0.9, 0.9")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$u_0(x; \\mu)$")
plt.show()

### Solve the Equation

In [ ]:
def solve_allen_cahn(Nt, Nx, mu):
    # discretize space and time
    x = np.linspace(0, 1, Nx)
    t = np.linspace(0, 1, Nt)

    dt = t[1] - t[0] # time step
    A = np.diag(-2*np.ones(Nx)) + np.diag(np.ones(Nx-1), 1) + np.diag(np.ones(Nx-1), -1)

    # initialize solution
    sol = np.zeros((Nt, Nx))
    sol[0] = u0(x, mu) # set time 0 to be initial condition

    # time step with forward Euler
    for t_i in range(1, Nt):
        sol[t_i] = sol[t_i-1] + dt*A@sol[t_i-1] + dt*f(sol[t_i-1])
    
    return sol

In [ ]:
Nt = 100
Nx = 100

# solve Allen-Cahn for three different parameters
mu = np.array([0.5, 0.1])
sol1 = solve_allen_cahn(Nt, Nx, mu)

mu = np.array([0.5, 0.5])
sol2 = solve_allen_cahn(Nt, Nx, mu)

mu = np.array([0.5, 0.9])
sol3 = solve_allen_cahn(Nt, Nx, mu)

# plot results
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
vmin = min(sol1.min(), sol2.min(), sol3.min())
vmax = max(sol1.max(), sol2.max(), sol3.max())

im1 = axes[0].imshow(sol1, vmin=vmin, vmax=vmax)
axes[0].set_title("$\\mu = (0.5, 0.1)$")

im2 = axes[1].imshow(sol2, vmin=vmin, vmax=vmax)
axes[1].set_title("$\\mu = (0.5, 0.5)$")

im3 = axes[2].imshow(sol3, vmin=vmin, vmax=vmax)
axes[2].set_title("$\\mu = (0.5, 0.9)$")

fig.colorbar(im1, ax=axes, orientation='vertical', fraction=.1)
plt.savefig("allen-cahn-solutions.pdf", bbox_inches="tight")
plt.show()

## Singular Values of Snapshot Matrix: Fast Decay!

In [ ]:
snapshots = np.concatenate((sol1, sol2, sol3))
U, S, V = np.linalg.svd(snapshots, full_matrices=False)
plt.scatter(np.arange(S.shape[0]), S)
plt.yscale('log')
plt.show()

## Let's compute a better POD basis

In [ ]:
# First, we solve the equation for some different parameters
Nt = 1000
Nx = 1000
snapshots = None
for mu1 in tqdm(np.linspace(0.1, 0.9, 4)):
    for mu2 in tqdm(np.linspace(0.1, 0.9, 4)):
        sol = solve_allen_cahn(Nt, Nx, np.array([mu1, mu2]))
        if snapshots is None:
            snapshots = sol
        else:
            snapshots = np.concatenate((snapshots, sol))

In [ ]:
U, S, V = np.linalg.svd(snapshots.T, full_matrices=False)

In [ ]:
plt.scatter(np.arange(S.shape[0])[:100], S[:100])
plt.yscale('log')
plt.show()

From this plot of the singular values, we see that we can take a small rank $r$, say $r=30$, to capture the data in our snapshot matrix. Let's now make a ROM to solve the Allen-Cahn equation.

In [ ]:
r = 30
U_r = U[:, :r]

Nt = 1000
Nx = 1000

# discretize space and time
x = np.linspace(0, 1, Nx)
t = np.linspace(0, 1, Nt)

dt = t[1] - t[0] # time step
A = U_r.T @ (np.diag(-2*np.ones(Nx)) + np.diag(np.ones(Nx-1), 1) + np.diag(np.ones(Nx-1), -1)) @ U_r

def solve_allen_cahn_rom(Nt, Nx, mu):
    # initialize solution
    sol = np.zeros((Nt, r))
    sol[0] = U_r.T @ u0(x, mu) # set time 0 to be initial condition

    # time step with forward Euler
    for t_i in range(1, Nt):
        sol[t_i] = sol[t_i-1] + dt*A@sol[t_i-1] + dt*U_r.T @ f(U_r @ sol[t_i-1])
    
    return sol

In [ ]:
snapshots_rom = None
for mu1 in tqdm(np.linspace(0.1, 0.9, 4)):
    for mu2 in tqdm(np.linspace(0.1, 0.9, 4)):
        sol = solve_allen_cahn_rom(Nt, Nx, np.array([mu1, mu2]))
        if snapshots_rom is None:
            snapshots_rom = sol
        else:
            snapshots_rom = np.concatenate((snapshots_rom, sol))

In [ ]:
print("Error on POD sample points")
print("Relative Error:", np.linalg.norm(snapshots - snapshots_rom @ U_r.T) / np.linalg.norm(snapshots), 
      "Absolute Error:", np.linalg.norm(snapshots - snapshots_rom @ U_r.T))

## Compute Error on New Samples

In [ ]:
rom_solutions = []
hf_solutions = []

for trial in tqdm(range(15)):
    mu1 = np.random.uniform(0.1, 0.9)
    mu2 = np.random.uniform(0.1, 0.9)
    sol_rom = solve_allen_cahn_rom(Nt, Nx, np.array([mu1, mu2]))
    sol_hf = solve_allen_cahn(Nt, Nx, np.array([mu1, mu2]))

    rom_solutions.append(sol_rom)
    hf_solutions.append(sol_hf)

In [ ]:
error_ = np.sqrt(np.sum((np.array(rom_solutions)@U_r.T - np.array(hf_solutions))**2))
norm_ = np.sqrt(np.sum(np.array(hf_solutions)**2))

print("Error on randomly sampled points")
print("Relative Error:", error_/norm_, "Absolute Error:", error_)